In [1]:
import re
import string

import razdel
import nltk
import pandas as pd
import pymorphy2
import pymystem3
from nltk.stem.snowball import SnowballStemmer
from rusenttokenize import ru_sent_tokenize

In [2]:
class BasicForm:                                     #основа или лемма
    def __init__(self, basic_form, frequency, form):
        self.basic_form = basic_form
        self.frequency = frequency
        self.forms = set([form])
        
    def match(self):
        for wordform in self.forms:
            if self.basic_form == wordform:
                return True
        return False

In [3]:
class PymorphyMorpher:    
    def __init__(self):
        self.morpher = pymorphy2.MorphAnalyzer()
        self.cash = {}
    def normal_form(self, word):
        if word in self.cash:
            lemma = self.cash[word]
        else:
                lemma = self.morpher.parse(word)[0].normal_form
                self.cash[word] = lemma
        return lemma

In [4]:
def remove_objects(text, pattern):
    text = re.sub(pattern, ' ', text)
    text = re.sub('  +', ' ',text)
    return text

In [5]:
def normalization(tokens, normalizator, mystem=False):
    normalized = {}
    for token in tokens:
        if mystem:
            try:
                basic_form = normalizator(token)
            except:
                basic_form = token['text']
            token = token['text']
        else:
            token = token.text
            basic_form = normalizator(token)
        if basic_form in normalized:
            normalized[basic_form].frequency += 1
            normalized[basic_form].forms.add(token)
        else:
            normalized[basic_form] = BasicForm(basic_form, 1, token)
    return normalized

In [47]:
def comparison(tag_value):
    d = {}
    counter_pymorphy = 0
    counter_mystem = 0
    summa = 0
    for lemma in lemmas_pymorphy:
        if lemma in lemmas_mystem:
            if tag_value in pymorhy_morpher.morpher.parse(lemma)[0].tag:
                summa += 1
                if lemmas_pymorphy[lemma].forms < lemmas_mystem[lemma].forms:
                    counter_mystem += 1
                if lemmas_pymorphy[lemma].forms > lemmas_mystem[lemma].forms:
                    counter_pymorphy += 1
    d = {'mystem увидел больше форм' : counter_mystem, 'pymorphy увидел больше форм' : counter_pymorphy, 'всего' : summa}
#    for key in d:
#        print(key, d[key])
    return d

Открываю текст

In [6]:
zhivago = 'zhivago.txt'
with open(zhivago, 'r', encoding='utf-8') as file:
    text = file.read()

In [7]:
text[:1000]

'<author><first-name>Борис</first-name><middle-name>Леонидович</middle-name><last-name>Пастернак</last-name></author>\n<book-title>Доктор Живаго</book-title>\n<annotation><p>«Доктор Живаго» - итоговое произведение Бориса Пастернака, книга всей его жизни. Этот роман принес его автору мировую известность и Нобелевскую премию, присуждение которой обернулось для поэта оголтелой политической травлей, обвинениями в «измене Родине» и в результате стоило ему жизни.</p>\n<p>«Доктор Живаго» - роман, сама ткань которого убедительнее свидетельствует о чуде, чем все размышления доктора и обобщения автора. Человек, который так пишет, бесконечно много пережил и передумал, и главные его чувства на свете  - восхищенное умиление и слезное сострадание; конечно, есть в его мире место и презрению, и холодному отстранению  - но не в них суть. Роман Пастернака  - оплакивание прежних заблуждений и их жертв; те, кто не разделяет молитвенного восторга перед миром, достойны прежде всего жалости. Перечитывать «До

1) Чищу от мусора

In [8]:
tag_pattern = "<.+?>"
xa0_pattern = "\xa0"
litres_pattern = 'Litres.ru|litres.ru|Downloader'
patterns = [tag_pattern, xa0_pattern, litres_pattern]

In [9]:
text = re.sub("\n", ' ', text)
for pattern in patterns:
    text = remove_objects(text, pattern)

In [10]:
text[:1000]

' Борис Леонидович Пастернак Доктор Живаго «Доктор Живаго» - итоговое произведение Бориса Пастернака, книга всей его жизни. Этот роман принес его автору мировую известность и Нобелевскую премию, присуждение которой обернулось для поэта оголтелой политической травлей, обвинениями в «измене Родине» и в результате стоило ему жизни. «Доктор Живаго» - роман, сама ткань которого убедительнее свидетельствует о чуде, чем все размышления доктора и обобщения автора. Человек, который так пишет, бесконечно много пережил и передумал, и главные его чувства на свете - восхищенное умиление и слезное сострадание; конечно, есть в его мире место и презрению, и холодному отстранению - но не в них суть. Роман Пастернака - оплакивание прежних заблуждений и их жертв; те, кто не разделяет молитвенного восторга перед миром, достойны прежде всего жалости. Перечитывать «Доктора Живаго» стоит именно тогда, когда кажется, что жить не стоит. Тогда десять строк из этого романа могут сделать то же, что делает любовь 

2) Разбиваю на предложения, удаляю пунктуацию и токенизирую

In [11]:
sentences = ru_sent_tokenize(text)

In [12]:
len(sentences)

14679

In [13]:
sentences[:10]

['Борис Леонидович Пастернак Доктор Живаго «Доктор Живаго» - итоговое произведение Бориса Пастернака, книга всей его жизни.',
 'Этот роман принес его автору мировую известность и Нобелевскую премию, присуждение которой обернулось для поэта оголтелой политической травлей, обвинениями в «измене Родине» и в результате стоило ему жизни.',
 '«Доктор Живаго» - роман, сама ткань которого убедительнее свидетельствует о чуде, чем все размышления доктора и обобщения автора.',
 'Человек, который так пишет, бесконечно много пережил и передумал, и главные его чувства на свете - восхищенное умиление и слезное сострадание; конечно, есть в его мире место и презрению, и холодному отстранению - но не в них суть.',
 'Роман Пастернака - оплакивание прежних заблуждений и их жертв; те, кто не разделяет молитвенного восторга перед миром, достойны прежде всего жалости.',
 'Перечитывать «Доктора Живаго» стоит именно тогда, когда кажется, что жить не стоит.',
 'Тогда десять строк из этого романа могут сделать т

In [14]:
punctuation = string.punctuation
in_ ='-'
out = '—'
table = str.maketrans(in_, out)
punctuation = punctuation.translate(table)
addition =  ['\ufeff', '…', '«', '»', '“', '”', '«', '»']
addition = ''.join(addition)
punctuation += addition

In [15]:
#text = text.strip(punctuation) так почему-то не сработало
text = text.lower()
deleting_table = str.maketrans('', '', punctuation)
text = text.translate(deleting_table)
text = remove_objects(text, ' - ')

In [16]:
text[:1000]

' борис леонидович пастернак доктор живаго доктор живаго итоговое произведение бориса пастернака книга всей его жизни этот роман принес его автору мировую известность и нобелевскую премию присуждение которой обернулось для поэта оголтелой политической травлей обвинениями в измене родине и в результате стоило ему жизни доктор живаго роман сама ткань которого убедительнее свидетельствует о чуде чем все размышления доктора и обобщения автора человек который так пишет бесконечно много пережил и передумал и главные его чувства на свете восхищенное умиление и слезное сострадание конечно есть в его мире место и презрению и холодному отстранению но не в них суть роман пастернака оплакивание прежних заблуждений и их жертв те кто не разделяет молитвенного восторга перед миром достойны прежде всего жалости перечитывать доктора живаго стоит именно тогда когда кажется что жить не стоит тогда десять строк из этого романа могут сделать то же что делает любовь в одном из стихотворений доктора жизнь ве

In [17]:
tokens = list(razdel.tokenize(text))

In [18]:
len(tokens)

162324

Повторяющиеся предложения:

In [19]:
print(len(sentences), len(set(sentences)))

14679 14490


In [20]:
sentences_dict = {}
for sentence in sentences:
    if sentence in sentences_dict:
        sentences_dict[sentence] += 1
    else:
        sentences_dict[sentence] = 1

In [21]:
for key in sentences_dict.keys():
    if sentences_dict[key] > 2:
        print(key, sentences_dict[key])

А? 7
Ха-ха-ха! 3
– Знаю. 4
– Да. 10
– А как же. 3
Не правда ли? 9
Конечно. 3
Где он? 3
Погоди. 3
Маркел! 3
Но дело не в этом. 3
Слушай. 3
Пойдем. 3
– Хорошо. 6
– Нет. 7
Да. 10
Дядя Воронюк! 3
– Разумеется. 4
– Еще бы. 3
Сеялки. 4
Молотилки». 4
– Я знаю. 3
Ну да! 3
Свеча горела на столе, Свеча горела. 3


Самый частотный токен длиннеe 6 символов:

In [22]:
tokens_dict = {}
for token in tokens:
    token = token.text
    if token in tokens_dict:
        tokens_dict[token] += 1
    else:
        tokens_dict[token] = 1

In [23]:
frequent_token = ('', 0)
for token in tokens_dict.keys():
    if len(token) > 6:
        if tokens_dict[token] > frequent_token[1]:
            frequent_token = (token, tokens_dict[token])

In [24]:
print(frequent_token)

('андреевич', 289)


3) Стемминг

In [25]:
stemmer = SnowballStemmer('russian')

In [26]:
stems = normalization(tokens, stemmer.stem)

Разные слова с одинаковой (по мнению стеммера) основой:

In [27]:
#тут конечно можно было использовать морфер, но задание про него идёт позже
for stem in stems.keys():
    if len(stems[stem].forms) > 10:
        pass
#        print(stem, '\n', stems[stem].forms) #слишком большой вывод

In [28]:
errors_a = ['сто', 'год', 'ран', 'осен']
for error in errors_a:
    print(error, '\n', stems[error].forms)

сто 
 {'стоивший', 'стоил', 'стоя', 'стоившую', 'стоим', 'сто', 'стоило', 'стоят', 'стоили', 'стой', 'стоишь', 'стоится', 'стою', 'стоит'}
год 
 {'года', 'годами', 'годам', 'годов', 'годился', 'годится', 'годом', 'год', 'годилось', 'году', 'годы', 'годившееся'}
ран 
 {'ранило', 'раннего', 'раннему', 'ранен', 'ранее', 'рань', 'раннею', 'ранить', 'ранней', 'раны', 'ране', 'ранила', 'ранних', 'ран', 'раннем', 'ранил', 'ранив', 'рано', 'ранний'}
осен 
 {'осенние', 'осени', 'осенила', 'осеннего', 'осенило', 'осень', 'осенью', 'осенний', 'осеннюю', 'осенняя', 'осеннее', 'осеннем'}


Слова, которые (ошибочно) не изменились после стемминга

In [29]:
for stem in stems.keys():
    stem = stems[stem]
    if len(stem.basic_form) > 4 and stem.match() and len(stem.forms) == 1:
        pass
#        print(stem.basic_form, '\n', stem.forms, '\n', stem.frequency) #слишком большой вывод

In [30]:
errors_b = ['спичек', 'льдом', 'подушечек', 'плывут']

In [31]:
for error in errors_b:
    print(error, stemmer.stem(error))

спичек спичек
льдом льдом
подушечек подушечек
плывут плывут


4) Стоп-слова из nltk

In [32]:
stop_words = nltk.corpus.stopwords.words('russian')

In [33]:
print(sorted(stop_words))

['а', 'без', 'более', 'больше', 'будет', 'будто', 'бы', 'был', 'была', 'были', 'было', 'быть', 'в', 'вам', 'вас', 'вдруг', 'ведь', 'во', 'вот', 'впрочем', 'все', 'всегда', 'всего', 'всех', 'всю', 'вы', 'где', 'да', 'даже', 'два', 'для', 'до', 'другой', 'его', 'ее', 'ей', 'ему', 'если', 'есть', 'еще', 'ж', 'же', 'за', 'зачем', 'здесь', 'и', 'из', 'или', 'им', 'иногда', 'их', 'к', 'как', 'какая', 'какой', 'когда', 'конечно', 'кто', 'куда', 'ли', 'лучше', 'между', 'меня', 'мне', 'много', 'может', 'можно', 'мой', 'моя', 'мы', 'на', 'над', 'надо', 'наконец', 'нас', 'не', 'него', 'нее', 'ней', 'нельзя', 'нет', 'ни', 'нибудь', 'никогда', 'ним', 'них', 'ничего', 'но', 'ну', 'о', 'об', 'один', 'он', 'она', 'они', 'опять', 'от', 'перед', 'по', 'под', 'после', 'потом', 'потому', 'почти', 'при', 'про', 'раз', 'разве', 'с', 'сам', 'свою', 'себе', 'себя', 'сейчас', 'со', 'совсем', 'так', 'такой', 'там', 'тебя', 'тем', 'теперь', 'то', 'тогда', 'того', 'тоже', 'только', 'том', 'тот', 'три', 'тут', 'ты

Стоп-слова, которые я бы сюда добавил:   
«также»: «тоже» есть в списке, а «также» нет, хотя они и не очень сильно различаются  
«почему»: синоним слова «зачем» которое есть в списке  
«ё-моё» или ещё какие-нибудь ругательные междометия  
«однако», «общем-то» и другие слова-паразиты   
«либо»  
«какое»

5) Лемматизация

pymorhy

In [34]:
pymorhy_morpher = PymorphyMorpher()
lemmas_pymorphy = normalization(tokens, pymorhy_morpher.normal_form)

mystem

In [35]:
mystem_morpher = pymystem3.Mystem()
lemmas_mystem_full = mystem_morpher.analyze(text)

In [36]:
lemmas_mystem = normalization(lemmas_mystem_full, lambda x: x['analysis'][0]['lex'], mystem=True)

Сравнение:

In [43]:
print('разные словофрмы соответсвуют одной лемме')
for lemma in lemmas_pymorphy:
    if lemma not in lemmas_mystem:
        pass
    else:
        if lemmas_pymorphy[lemma].forms != lemmas_mystem[lemma].forms:
            pass
#            print('pymorphy:', lemma, lemmas_pymorphy[lemma].forms)  #слишком большой вывод
#            print('mystem:', lemma, lemmas_mystem[lemma].forms)
#            print('\n')

разные словофрмы соответсвуют одной лемме


In [44]:
print('нет в майстем')
for lemma in lemmas_pymorphy:
    if lemma not in lemmas_mystem:
        pass
#        print(lemma)             #слишком большой вывод

нет в майстем


In [54]:
print('нет в пайморфи')
not_in_pymorphy = []
for lemma in lemmas_mystem:
    if lemma not in lemmas_pymorphy:
        not_in_pymorphy.append(lemma)
#        print(lemma)               #слишком большой вывод

нет в пайморфи


1. Майстем лучше разрешает многозначность, потому что использует контекст, например для слов «полк» и «полка» результат такой, (конечно я не посмотрел все употребления этих слов в тексте, но «Доктор Живаго» в том числе и про войну):

In [46]:
print(lemmas_mystem['полк'].forms)
print(lemmas_mystem['полка'].forms)
print('\n')
print(lemmas_pymorphy['полк'].forms)
print(lemmas_pymorphy['полка'].forms)

{'полком', 'полка', 'полк', 'полков', 'полку'}
{'полке', 'полками', 'полкой', 'полок'}


{'полк'}
{'полок', 'полками', 'полке', 'полкой', 'полка', 'полку'}


2. Майстем лучше обрабатывает глаголы (cводит больше(по вложению) форм к одному глаголу, будем считать, что это хорошо, потому что неодназначность майстем тоже разрешает лучше):

In [48]:
comparison('INFN')

{'mystem увидел больше форм': 1027,
 'pymorphy увидел больше форм': 57,
 'всего': 3635}

И вообще для всех тегов оперкорпоры майстем сводит больше форм к одной лемме

In [49]:
all_tags = set()
for lemma in lemmas_pymorphy:
    current_tags = pymorhy_morpher.morpher.parse(lemma)[0].tag
    current_tags = str(current_tags).split(',')
    current_tags_clear = []
    for tag in current_tags.copy():
        tag = tag.split()
        for tag_2 in tag:
            current_tags_clear.append(tag_2)
    current_tags_clear = set(current_tags_clear)
    all_tags = all_tags.union(current_tags_clear)

In [50]:
df = pd.DataFrame(all_tags, columns=['tag'])

In [51]:
for key in comparison('CONJ'):
    df[key] = df['tag'].apply(lambda x: comparison(x)[key])

In [52]:
df

,tag,mystem увидел больше форм,pymorphy увидел больше форм,всего
0,actv,2,0,6
1,LATN,0,0,26
2,Orgn,0,0,8
3,GRND,2,0,9
4,gent,36,11,181
...,...,...,...,...
78,Pltm,4,3,83
79,sing,359,207,9994
80,loct,2,0,21
81,nomn,287,190,9023


In [60]:
df.where(df['mystem увидел больше форм'] > df['pymorphy увидел больше форм']).shape

(83, 4)

Пайморфи пропустил много (на глаз) имён собственных:

In [55]:
names = ['цезарь', 'брусилов', 'александровна', 'памфилова', 'генц']
for name in names:
    if name in not_in_pymorphy:
        print(name)

цезарь
брусилов
александровна
памфилова
генц


Получается майстем лучше